In [14]:
import urllib.request
import re
from bs4 import BeautifulSoup
from lxml import html
import pandas as pd
import json
import requests


def range_price(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    lowend = soup.find_all("span", {"class":"value", "itemprop":"lowprice", "content": True})
    highend = soup.find_all("span", {"class":"value", "itemprop":"highprice", "content": True})
    quantityList = []
    rangePrice = ""

    for i in lowend:
        rangePrice+=(i["content"])
        rangePrice+=" - "
        break

    for i in highend:
        rangePrice+=(i["content"])
        break

    return rangePrice



def scrape_rifle_helper(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
    url+= "=undefined&start=0&sz=100"
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    ammo = soup.find_all("div", class_="pdp-link")
    ammoPrice = soup.find_all("div",  class_="tile-body")
    names = []
    prices = []
    actual_prices = []
    possibleCaliber = soup.find_all("li", class_="filter-value")
    caliber = possibleCaliber[0].text
    
    for i in ammo:
        names.append(i.find("span").text)

    for i in ammoPrice:
        # if the product has a range of prices go here
        if '<span class="range">' in str(i):
            x = i.find("a", {"class": "link", "href":True})
            prices.append(x["href"])

        # if the product has a slashed price, go here
        else:
            x = (i.find("span", class_="sales"))
            prices.append(x.find("span", {"class":"value", "content": True}))


    # gather product detail links
    soup_links = soup.find_all("div", class_="pdp-link")
    links = []
    for i in soup_links:
        aLink = re.findall("(?P<url>https?://[^\s]+)", str(i))
        x = (str(aLink[0]).replace('">', "").replace('"', ""))
        links.append(x)



    # add prices in the prices list
    for i in prices:
        if "http" in i:
            rp = range_price(i)
            actual_prices.append(rp)

        else:
            htmlPriceTag = str(i)
            splitHtmlPriceTag = htmlPriceTag.split(" ")
            p =  splitHtmlPriceTag[2].strip('content="')
            actual_prices.append(str(p))
   
    try:
        productList = pd.DataFrame(
            {"name": names, "price": actual_prices, "link": links, "caliber": caliber.replace("\n\n", "").replace("\n\n\n", "").replace("\n", "")}
        )
        return productList

    except:
        print("something went wrong")

In [15]:
link = "https://www.cheaperthandirt.com/shop-by?cgid=79&searchBy=Caliber"

def scrape_rifle(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 \
        (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }
        
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "lxml")
    sections = soup.find_all("div", class_="brand-link-list")

    # get caliber number set
    caliberNumbers = []
    x = soup.find_all('div', class_="refinements")
    for i in x:
        cn = i.find_all("div", {"id": True})
        for j in cn:
            caliberNumbers.append((j["id"]))

    dirtyLinks = []  
    calibers = []
    for i in sections:
        dirtyLinks.append(i.find_all("a"))
        x = (i.find_all("a"))
        for j in x:
            c = (j.find("span").text)
            calibers.append(c)


    caliberNumberDF = pd.DataFrame(
        {"number": caliberNumbers, "section": dirtyLinks}
    )
    
    arrayOfLinks = []
    index = 0
    for i in caliberNumberDF["section"]:
        linkToCaliberList = []  # links
        for j in i:
            aLink = re.findall("(?P<url>https?://[^\s]+)", str(j))
            arrayOfLinks.append(str(aLink[0]).replace('">', ""))

    arrayOfDicts = []
    aNameList = []
    aPriceList = []
    aLinkList = []
    aCaliberList = []
    
    for i in arrayOfLinks:
        productDict = {"caliber": '', "names": '', "price": '', "link": ''}
        productDF = scrape_rifle_helper(i)
        for name in productDF["name"]:
            aNameList.append(name)

        for price in productDF["price"]:
            aPriceList.append(price)

        for link in productDF["link"]:
            aLinkList.append(link)
    
        for caliber in productDF["caliber"]:
            aCaliberList.append(caliber)

    ammo_data = {"type":"rifle", "ammo_results": []}
        

    for i in range(0, len(aNameList)):
        x = {"name": aNameList[i], "price": aPriceList[i], "caliber": aCaliberList[i], "link": aLinkList[i]}
        ammo_data["ammo_results"].append(x)

        
    return ammo_data

In [16]:
ammo = scrape_rifle(link)
from pathlib import Path
import os, shutil, glob
rel = "jsonFiles/cheaperThanDirt/ammo"
cwd = Path.cwd()
currentPath = str(Path.cwd())+""
aCount = 0
savePath = str(cwd).replace("pyScripts/cheaperThanDirt", rel)


swiftTransferDict = {"results": []}

with open(os.path.join(savePath, f"CTDRifleAmmo.json"), "w") as fp:
    json.dump(ammo, fp)



rel = 'crispy-computing-app/crispy-computing-app/jsonFiles/cheaperThenDirt/ammo'
cwd = Path.cwd()
currentPath = str(Path.cwd())+''
savePath = str(cwd).replace("pyScripts/cheaperThanDirt", rel)
index = 0
with open(os.path.join(savePath, f"CTDRifleAmmo.json"), "w") as fp:
    json.dump(ammo, fp)

https://www.cheaperthandirt.com/.22-250-remington/ammunition/rifle-ammo/?brand=.22-250Remington
https://www.cheaperthandirt.com/.223-remington/ammunition/rifle-ammo/?brand=.223Remington
https://www.cheaperthandirt.com/.224-valkyrie/ammunition/rifle-ammo/?brand=.224Valkyrie
https://www.cheaperthandirt.com/.25-06-remington/ammunition/rifle-ammo/?brand=.25-06Remington
https://www.cheaperthandirt.com/.257-weatherby-magnum/ammunition/rifle-ammo/?brand=.257WeatherbyMagnum
https://www.cheaperthandirt.com/.270-weatherby-magnum/ammunition/rifle-ammo/?brand=.270WeatherbyMagnum
https://www.cheaperthandirt.com/.300-aac-blackout/ammunition/rifle-ammo/?brand=.300AACBlackout
https://www.cheaperthandirt.com/.30-378-weatherby-magnum/ammunition/rifle-ammo/?brand=.30-378WeatherbyMagnum
https://www.cheaperthandirt.com/.308-winchester/ammunition/rifle-ammo/?brand=.308Winchester
https://www.cheaperthandirt.com/.30-carbine/ammunition/rifle-ammo/?brand=.30Carbine
https://www.cheaperthandirt.com/.33-nosler/amm